In [107]:
import csv
import string
import pandas as pd
from nltk.tokenize import word_tokenize

pd.set_option("display.max_colwidth", 250)

dataset = pd.read_csv("SMSSpamCollection", sep="\t", header=None)
dataset.columns = ["labels", "sms_msgs"]
col1 = dataset.groupby("labels") 
ratio = col1.count().values[0]/col1.count().values[1]

def remove_punctuation(text):
    without_punct = [t for t in text if t not in string.punctuation]
    return "".join(without_punct)

In [43]:
from nltk.corpus import stopwords

dataset["clean_msg"] = dataset["sms_msgs"].apply(lambda msg: remove_punctuation(msg))
dataset["tokenized_msg"] = dataset["clean_msg"].apply(lambda msg: word_tokenize(msg.lower()))

# removing stopwords
def remove_sw(msg):
    words = list()
    for m in msg:
        if m not in stopwords.words("english"):
            words.append(m)
    return words

dataset["without_sw"] = dataset["tokenized_msg"].apply(lambda msg: remove_sw(msg))

In [46]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

def stem_msg(msg):
    words = list()
    for m in msg:
        words.append(ps.stem(m))
    return words

dataset["stemmed_msg"] = dataset["without_sw"].apply(lambda msg: stem_msg(msg))

In [91]:
from nltk import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_msg(sent=None):
    if sent is None:
        sent = list()
    else:
        sent = sent
    new_sent = list()
    for s in sent:
        new_sent.append(lemmatizer.lemmatize(s))
    return new_sent

dataset["lemmatized_msg"] = dataset["without_sw"].apply(lambda msg: lemmatize_msg(msg))

In [125]:
# pre-processing 
def text_clean(msg):
    msg =  remove_punctuation(msg)
    msg = word_tokenize(msg)
    msg = remove_sw(msg)
    msg = stem_msg(msg)
    #msg = lemmatize_msg(msg)
    return msg

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf =  TfidfVectorizer(analyzer=text_clean)
# cv =  CountVectorizer(ngram_range(3,3)) # ngram 
x = tfidf.fit_transform(dataset["sms_msgs"])
dataFrame = pd.DataFrame(x.toarray(), columns=cv.get_feature_names())
print(dataFrame.shape)

(5572, 8401)
